# ArmourEye LLM + RAG Setup (Colab)

This notebook sets up the FastAPI inference server with Mistral 7B + RAG in Google Colab.

**Requirements:**
- GPU runtime (Runtime → Change runtime type → GPU)
- Google Drive mounted with ArmourEye project


# 1. Install dependencies (once per new Colab session)


In [ ]:
!pip install fastapi uvicorn langchain-community langchain-chroma sentence-transformers \
transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 123.9 MB/s eta 0:00:0

# 2. Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 3. Set env vars and start uvicorn (only once; re-run if the session resets)


In [ ]:
import os

BASE_DRIVE_PATH = "/content/drive/<Folder Path>"
os.environ["MISTRAL_MODEL_PATH"] = os.path.join(BASE_DRIVE_PATH, "<Model Path>")
os.environ["VECTORSTORE_DIR"] = os.path.join(BASE_DRIVE_PATH, "Vector Dataset Path")

# 4. Check if Vector Dataset, Model and server.py Exists


In [ ]:
import os
os.path.isdir("/content/drive/MyDrive/<Folder Path>/ArmourEye-main/backend/ai/models/mistral")

In [ ]:
import os
os.path.isdir("/content/drive/MyDrive/<Folder Path>/ArmourEye-main/backend/ai/vectorstore/chroma_db_v2")

True

In [ ]:
import os
os.path.exists("/content/drive/MyDrive/Folder Path/ArmourEye-main/backend/ai/inference/server.py")

True

**To check for syntax error**

In [ ]:
!python -m py_compile /content/drive/MyDrive/TESTING/ArmourEye-main/backend/ai/inference/server.py

# 5. To Start UVICORN

In [ ]:
%cd /content/drive/MyDrive/TESTING/ArmourEye-main
!nohup uvicorn backend.ai.inference.server:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &

/content/drive/MyDrive/TESTING/ArmourEye-main


**CHECK THE NEXT CODE TO SEE IF UVICORN IS RUNNING< THEN ONLY DO THE NEXT COUPLE OF CODES**

In [ ]:
!tail -n 20 uvicorn.log

2025-11-29 05:10:27.652567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764393027.674185    3458 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764393027.680894    3458 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764393027.697864    3458 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764393027.698003    3458 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764393027.698052    3458 computation_placer.cc:177] computation placer alr

# 6. Start cloudflared (foreground once to get URL, then background)


**ONLY RUN THE CODE BELOW IF YOU DELETE RUNTIME OR IF YOU ARE DOING THIS THE FIRST TIME**

In [ ]:
!curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
!chmod +x cloudflared

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 39.3M  100 39.3M    0     0  9234k      0  0:00:04  0:00:04 --:--:--  116M


# 7. To Start Cloudflare

In [ ]:
!nohup ./cloudflared tunnel --no-autoupdate --url http://localhost:8000 > cloudflared.log 2>&1 &

# 8. To Get Cloudflare Link

In [ ]:
!grep -m1 -o 'https://[a-zA-Z0-9.-]*trycloudflare.com' cloudflared.log

https://anna-satisfied-tray-satin.trycloudflare.com


**COPY THE URL AND RUN THIS IN BACKEND POWERSHELL
node ai/update_remote_url.js PUT THE URL HERE**

In [ ]:
!tail -n 20 cloudflared.log

2025-11-29T05:16:35Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-29T05:16:40Z INF +--------------------------------------------------------------------------------------------+
2025-11-29T05:16:40Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-29T05:16:40Z INF |  https://anna-satisfied-tray-satin.trycloudflare.com                                       |
2025-11-29T05:16:40Z INF +--------------------------------------------------------------------------------------------+
2025-11-29T05:16:40Z INF Cannot determine default configuration path. No file [config.yml config.yaml] in [~/.cloudflared ~/.cloudflare-warp ~/cloudflare-warp /etc/cloudflared /usr/local/etc/cloudflared]
2025-11-29T05:16:40Z INF Version 2025.11.1 (Checksum 991dffd8889ee9f0147b6b48933da9e4407e68ea8c6d984f55fa2d3db4bb431d)
2025-11-29T05:16:40Z INF GOOS: linux, GOVersion: go1.24.9, GoArch: amd64
2025-11-29T05:16:40Z INF Settings: map[ha-conne

# TO KILL TUNNELS

In [ ]:
!pkill -f cloudflared
!pkill -f uvicorn